In [ ]:
import pandas as pd
import numpy as np
import json, csv
import itertools as it
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm.notebook import tqdm
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en')
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:

# Import files from google drive
from google.colab import drive
drive.mount('/content/gdrive',force_remount=False)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#change directory
os.chdir('/content/gdrive/My Drive/product_workshop/data')
os.listdir()

['review_corpus.txt',
 'unigram_sentences_all.txt',
 'bigram_sentences_all.txt',
 'trigram_sentences_all.txt',
 'trigram_transformed_reviews_all.txt',
 'trigram_dict_all.dict',
 'trigram_bow_corpus_all.mm.index',
 'trigram_bow_corpus_all.mm',
 'lda_model_all',
 'lda_model_all.expElogbeta.npy',
 'lda_model_all.state',
 'lda_model_all.id2word',
 'ratings_sample.csv',
 'sampled_reviews.csv',
 'sampled_reviews_adj.csv']

In [ ]:
import sys
csv.field_size_limit(sys.maxsize)

131072

In [ ]:
with open('sampled_reviews_adj.csv') as csv_file:
  csv_reader = csv.reader(csv_file, delimiter=',')
  print(next(csv_reader))
  print(next(csv_reader))
  print(next(csv_reader))
  print(next(csv_reader))
  print(next(csv_reader))

['', 'overall', 'verified', 'reviewTime', 'reviewerID', 'asin', 'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote', 'image', 'title', 'char_count', 'sum_reviews', 'review_adjectives']
['0', '5.0', 'False', '06 19, 2000', 'A9L3BUKDG2XNM', 'B000068MBW', "{'Format:': ' VHS Tape'}", 'Amazon Customer', 'you will watch this movie and be unable to believe that it was made for television it is so well done a labor of love for director jim abrahams whose own son suffered from epilepsy helped by alternative medicine this is the story of a farmer ward and his wife the ever wonderful streep making a rare television appearance whose youngest son robbie adkins developes epilepsy michael yarmush also fares well in his role as robbies older brother frank telling of a childs illness and its profound effect on a family', 'Realistic and Gripping!', '961372800', '10.0', '', '"...First Do No Harm"', '532.0', '80.0', "['unable', 'own', 'alternative', 'wonderful', 'rare', 'profound']"

In [ ]:
%%time
df_cleaned = pd.read_csv('sampled_reviews_adj.csv', converters={'review_adjectives': eval})

CPU times: user 4.24 s, sys: 384 ms, total: 4.63 s
Wall time: 4.85 s


In [ ]:
print(f"There are {df_cleaned.shape[0]} rows")
print(f"There are {len(df_cleaned.asin.unique())} movies being reviewed")

There are 134370 rows
There are 14080 movies being reviewed


In [ ]:
df_cleaned.shape

(134370, 17)

In [ ]:
df_cleaned['overall'].value_counts()

5.0    69573
4.0    28539
3.0    16083
1.0    11124
2.0     9051
Name: overall, dtype: int64

In [ ]:
df_cleaned = df_cleaned[df_cleaned['overall'] > 3]

In [ ]:
print(f"There are {df_cleaned.shape[0]} rows left")
print(print(f"There are {len(df_cleaned.asin.unique())} movies being reviewed"))

There are 98112 rows left
There are 13888 movies being reviewed
None


In [ ]:
data_words = df_cleaned.review_adjectives
data_words[0]

['unable', 'own', 'alternative', 'wonderful', 'rare', 'profound']

In [ ]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.ldamulticore import LdaMulticore

In [ ]:
id2word = corpora.Dictionary(data_words)

In [ ]:
corpus = [id2word.doc2bow(text) for text in data_words]

In [ ]:
print(corpus[:2])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)], [(6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1)]]


In [ ]:
%%time
lda_model = LdaMulticore(corpus=corpus, 
                         id2word=id2word,
                         num_topics=50,
                         random_state=100,
                         chunksize=100,
                         passes=10,
                         per_word_topics=True,
                         workers = 3)

CPU times: user 24min 34s, sys: 2min 7s, total: 26min 42s
Wall time: 27min 30s


In [ ]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 2.7MB/s 
     |████████████████████████████████| 552kB 16.5MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=c3afcecd179b66eee6bcde7a734b31ba9b847b8b5b04e566abe84fbcda7b3c98
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=1ac8b9af94f0070c2f0c26e16d80315bde076dfce82a7a959fd89e113bc20bd7
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [ ]:
import pyLDAvis
import pyLDAvis.gensim
import warnings
import pickle

In [ ]:
def explore_topic(topic_number, topn=25):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
        
    print('{:20} {}'.format(u'term', u'frequency') + u'\n')

    for term, frequency in lda_model.show_topic(topic_number, topn=25):
        print(u'{:20} {:.3f}'.format(term, round(frequency, 3)))

In [ ]:
explore_topic(topic_number=2)

term                 frequency

old                  0.383
classic              0.227
martial              0.031
intelligent          0.027
suspenseful          0.020
secret               0.020
sorry                0.016
magical              0.016
wont                 0.012
colorful             0.012
other                0.011
capable              0.010
randy                0.010
andy                 0.008
leslie               0.006
sam                  0.005
wealthy              0.005
philosophical        0.005
evident              0.005
delicious            0.004
learning             0.004
marvel               0.004
nephew               0.004
cynical              0.004
twin                 0.004


In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=corpus, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

Streaming output truncated to the last 5000 lines.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Us


Perplexity:  -8.416940251071455

Coherence Score:  nan


In [ ]:
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

In [ ]:
pyLDAvis.display(LDAvis_prepared)